# Generate S2Cell IDs from a KML file Example

This codelab will guide you through an example of how to generate a list of S2 Cell IDs that cover a specific area defined in a KML file.

Install dependencies.

In [1]:
%pip install s2sphere
%pip install geopandas
%pip install fiona

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Import and configure libraries.

In [2]:
import geopandas as gpd
import s2sphere as s2
import fiona

fiona.drvsupport.supported_drivers['KML'] = 'rw'

Define functions for extracting S2Cell ID.

In [3]:
def fetch_polygons_in_kml(kml_path: str) -> list[list[tuple[float, float]]]:
    """Fetches list of polygons for all geometries in kml file.

    Args:
      kml_path: path to kml file

    Returns:
      returns list of lat lng polygons for all geometries in kml file
    """
    gdf = gpd.read_file(kml_path, driver='KML')
    polygon_vertices = []
    for i in range(gdf.shape[0]):
        exterior_coords = []
        geometry = gdf.iloc[i].geometry
        if geometry.geom_type == 'MultiPolygon':
            # Iterate over each Polygon in the MultiPolygon
            for polygon in geometry.geoms:
                exterior_coords += list(polygon.exterior.coords)
        elif geometry.geom_type == 'Polygon':
            exterior_coords += list(geometry.exterior.coords)
        else:
            continue  # Skip non-polygon geometries

        polygon_vertices.append(
            [(float(coord[1]), float(coord[0])) for coord in exterior_coords]
        )
    return polygon_vertices


def get_covering_s2_cells(polygon, level: int):
  """Gets a list of S2 cell IDs of a given level that cover the bounding box of a polygon.

  A visualization of this covering can be seen at http://s2geometry.io/devguide/examples/coverings.html

  Args:
    lat: list of coordinates i.e. [lat, lng] of a polygon or the corner
      coordinates of a rectangle.
    level: level at which s2cells are to be generated

  Raises:
    Exception: no s2 cells are found i.e. if illegal polygon is passed.
  Returns:
    s2_cover: list of s2cell ids(ints) tht cover the whole polygon.
  """

  lat_lngs = [s2.LatLng.from_degrees(lat, lng) for lat, lng in polygon]

  lats = [l.lat().radians for l in lat_lngs]
  lngs = [l.lng().radians for l in lat_lngs]

  min_lat = min(lats)
  max_lat = max(lats)
  min_lng = min(lngs)
  max_lng = max(lngs)

  s2_min_pt = s2.LatLng.from_radians(min_lat, min_lng)
  s2_max_pt = s2.LatLng.from_radians(max_lat, max_lng)
  rect = s2.LatLngRect.from_point_pair(s2_min_pt, s2_max_pt)

  coverer = s2.RegionCoverer()
  coverer.min_level = level
  coverer.max_level = level
  covering = coverer.get_covering(rect)
  if not covering:
    print('No s2 cover for polygon %s', polygon)
  s2_cell_ids = [cell.id() for cell in covering]

  return s2_cell_ids


def get_s2_cells_covering_kml(polygons_vertices: list[list[tuple[float, float]]], s2_cell_level: int) -> set[str]:
  """

  Args:
    polygons_vertices: list of polygons
    s2_cell_level: s2 cell level

  Returns: s2 cells at specified level for a region

  """
  s2_cell_ids_covering_kml = set()
  for polygon in polygons_vertices:
    s2_cell_ids_covering_polygon = get_covering_s2_cells(
        polygon=polygon, level=s2_cell_level
    )
    s2_cell_ids_covering_kml.update(s2_cell_ids_covering_polygon)
  return s2_cell_ids_covering_kml


Example function usage.

In [4]:
# ALU API requires S2Cell IDs to be level 13.
s2_cell_level = 13

# Example KML file
kml_file = 'https://raw.githubusercontent.com/datameet/Municipal_Spatial_Data/master/Chennai/CMA.kml'

polygons_vertices = fetch_polygons_in_kml(kml_file)
s2_cells_uint_64_format = get_s2_cells_covering_kml(polygons_vertices, s2_cell_level)
s2cell_ids = list(s2_cells_uint_64_format)

s2cell_ids

[4202494021597134848,
 4202494227755565056,
 4202494262115303424,
 4202494055956873216,
 4202494193395826688,
 4202494296475041792,
 4202494330834780160,
 4202494365194518528,
 4202494399554256896,
 4202494433913995264,
 4202494468273733632,
 4202494502633472000,
 4202494536993210368,
 4202494571352948736,
 4202494605712687104,
 4202575832134189056,
 4202496667296989184,
 4202690181343477760,
 4202537039989571584,
 4202558892783173632,
 4202574629543346176,
 4202569166344945664,
 4202563703146545152,
 4202558239948144640,
 4202536387154542592,
 4202527453622566912,
 4202529652645822464,
 4202531851669078016,
 4202570368935788544,
 4202556109644365824,
 4202573701830410240,
 4202691383934320640,
 4202559442538987520,
 4202545183247564800,
 4202534050692333568,
 4202523708411084800,
 4202524911001927680,
 4202526113592770560,
 4202527316183613440,
 4202523055576055808,
 4202524258166898688,
 4202518794968498176,
 4202496117541175296,
 4202497217052803072,
 4202530923956142080,
 420255731